In [ ]:
import psycopg2
import pandas as pd
from datetime import datetime


Step 1: Connect to RisingWave

In [ ]:
import psycopg2

conn = psycopg2.connect(
    host="rwc-g1iod2ng1pe9jb0hpk22vhqj2s-my-project.prod-aws-euno1-eks-a.risingwave.cloud",
    port=4566,
    user="jd",
    password="Degirk23@",
    dbname="dev",
    sslmode="require",
)
cursor = conn.cursor()
print("✅ Connected to RisingWave!")


✅ Connected to RisingWave!


Step 2: Insert data manually with real-time timestamps


In [ ]:
def insert_manual():
    try:
        order_id = int(input("🆔 Order ID: "))
        customer_name = input("👤 Customer Name: ")
        restaurant_name = input("🍽️ Restaurant Name: ")
        item_name = input("🥡 Item Name: ")
        delivery_status = input("🚚 Delivery Status (Preparing/Delivered/Cancelled): ")
        rating = float(input("⭐ Rating (0 to 5): "))

        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Real-time timestamp

        cursor.execute("""
            INSERT INTO food_orders
            (order_id, customer_name, restaurant_name, item_name, order_time, delivery_status, delivery_time, rating)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
        """, (order_id, customer_name, restaurant_name, item_name,
              current_time, delivery_status, current_time, rating))
        conn.commit()
        print("✅ Record inserted!\n")

    except Exception as e:
        print(f"❌ Error: {e}")

Step 3: Upload from CSV or Excel (with real-time timestamp override)

In [ ]:
def insert_from_csv_or_excel():
    from google.colab import files
    uploaded = files.upload()

    for fn in uploaded.keys():
        print(f"📄 Processing file: {fn}")

        if fn.endswith('.csv'):
            df = pd.read_csv(fn)
        elif fn.endswith(('.xlsx', '.xls')):
            df = pd.read_excel(fn)
        else:
            print("❌ Unsupported file type. Please upload CSV or Excel.")
            return

        for i, row in df.iterrows():
            try:
                now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Real-time timestamp
                cursor.execute("""
                    INSERT INTO food_orders
                    (order_id, customer_name, restaurant_name, item_name, order_time, delivery_status, delivery_time, rating)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
                """, (
                    int(row['order_id']),
                    row['customer_name'],
                    row['restaurant_name'],
                    row['item_name'],
                    now,  # use current time
                    row['delivery_status'],
                    now,  # use current time
                    float(row['rating']),
                ))
                conn.commit()
            except Exception as e:
                print(f"❌ Failed to insert row {i}: {e}")
        print("✅ All data inserted.\n")


Step 4: Simple Menu (Only two options)

In [ ]:
print("\n📋 Choose an option:")
print("1️⃣ Insert data manually")
print("2️⃣ Upload CSV or Excel to RisingWave")

choice = input("Enter your choice (1/2): ")

if choice == '1':
    insert_manual()
elif choice == '2':
    insert_from_csv_or_excel()
else:
    print("❌ Invalid choice.")



📋 Choose an option:
1️⃣ Insert data manually
2️⃣ Upload CSV or Excel to RisingWave
Enter your choice (1/2): 2
